## Sentiment Analysis

In [23]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import sys

host = '127.0.0.1'
port = "3306"
user = 'root'
password = 'ugm2018'
database = 'gdelt2'

table_name = 'events_ID'
fips_country_code = 'ID'
country_code = 'IDN'
# IDN, MYS, THA, SGP, PHL

conn_str = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, database);           
engine = create_engine(conn_str)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

def execute(sql):
    return engine.execute(statement=sql)

In [15]:
def get_sentiment(avgTone):
    s = float(avgTone)
    if (s > 1):
        return 1
    elif (s < -1):
        return -1
    else:
        return 0

In [ ]:
import os

corpus = []

for subdir, dirs, files in os.walk('./EconomicNews/IDN2/'):
    for file in files:
        filepath = file

        if filepath.endswith(".txt"):
            global_id = filepath[:-4]
           
            f = open("./EconomicNews/IDN2/"+filepath,"r", encoding='UTF-8')
            text = f.read()
            
            avgTone = run("""
            SELECT AvgTone from events_ID WHERE GLOBALEVENTID={}
            """.format(global_id))

            sentiment = get_sentiment(avgTone.iloc[0,0])                       
            
            corpus.append({'text': text ,'sentiment': sentiment, 'AvgTone': avgTone.iloc[0,0]})

In [81]:
df_corpus = pd.DataFrame(corpus)

In [83]:
df_corpus.sample(5)

,AvgTone,sentiment,text
223,1.66112956810632,1,India is home to highest number of generous pe...
180,0.86956521739131,0,The participation of Indonesia at the ITB Asia...
185,3.27868852459016,1,President of the Republic of Korea (RoK) Moon ...
34,-4.07885791978246,-1,TWELVE stragglers of the Maute Group were kill...
442,2.61154379331308,1,Leading global investment firm KKR today annou...


## Training

In [84]:
from sklearn.cross_validation import train_test_split

X = df_corpus.text
y = df_corpus.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [140]:
from nltk.stem.porter import PorterStemmer
import re

porter_stemmer = PorterStemmer()

def stemming_tokenizer(str_input):
    words = re.sub(r"[^a-z]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words


In [179]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

# token_pattern='[a-z]+',
vectorizer = TfidfVectorizer(analyzer=u'word', tokenizer=stemming_tokenizer, stop_words='english', max_df=0.7, lowercase=True)
X = vectorizer.fit_transform(X_train)

X_train_dtm = vectorizer.transform(X_train)

In [180]:
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

,aacyunk,aak,aal,ab,abandon,abang,abat,abbott,abc,abdul,...,zime,zo,zon,zone,zuckerberg,zulfikar,zulkifli,zumpango,zundin,zxr
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.040330,0.0,0.000000,0.0,0.000000,0.0,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [181]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [182]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 2.71 ms, sys: 0 ns, total: 2.71 ms
Wall time: 2.64 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Test

In [183]:
X_test_dtm = vectorizer.transform(X_test)

y_pred_class = nb.predict(X_test_dtm)

In [184]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.8

In [185]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[25,  0,  0],
       [ 4,  0,  1],
       [ 4,  0, 11]])

In [194]:
test_df = pd.DataFrame(
    {'text': X_test,
     'sentiment': y_test     
    })

predicts = []
for text in test_df['text'].values:
    dtm = vectorizer.transform([text])
    sn = nb.predict(dtm)[0]
    predicts.append(sn)

test_df['prediction'] = predicts
test_df['correct'] = test_df['sentiment'] == test_df['prediction']

In [213]:
test_df

,text,sentiment,prediction,correct
96,Indonesian vice police chief visits Australia ...,1,-1,False
171,KUALA LUMPUR (Nov 3): Based on corporate annou...,0,-1,False
353,0 Fireworks factory fire in Indonesia kills 47...,-1,-1,True
360,"JAKARTA, Indonesia — A police official says 47...",-1,-1,True
207,"HE already has a Master’s degree, a PhD, is a ...",1,1,True
107,KONTAN.CO.ID - JAKARTA. The central government...,-1,-1,True
18,Asean's push to loosen protectionist chains th...,1,1,True
274,© AP\n\n\n\nMore than 40 people were killed af...,-1,-1,True
60,"Based on data from Statistics Indonesia (BPS),...",1,1,True
238,Jakarta. Foreign Affairs Minister Retno Marsud...,-1,-1,True
